# Final stage of HELP data processing

This notebook performs collates the final output files ready for writing to csv for ingestion to a VO server

- Take DR1 masterlist suffixes from overview table
- Find dmu32 full table names and write to a file

In [1]:
from astropy.table import Table, Column
from astropy import units as u
import numpy as np

from pymoc import MOC

from herschelhelp_internal.masterlist import find_last_ml_suffix

import os

import humanfriendly

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


## The definition of HELP DR1
Here we take the DR1 definition from dmu1. We then propagate those suffixes through to dmu32. Note that this does not include a specification of how the masterlist made its way through the whole HELP process.

In [2]:
dr1 = Table.read("../dmu1/dr1_overview.fits")

In [3]:
dr1.show_in_notebook()

idx,field,dr1_suffix,last_suffix,objects,area_sq_degrees,file_size_bytes,file_size_readable
0,AKARI-NEP,20180215,20180215,531746,9.0,348848640,348.85 MB
1,AKARI-SEP,20180221,20180221,844172,9.0,538606080,538.61 MB
2,Bootes,20180216,20180227,3367490,11.0,4310429760,4.31 GB
3,CDFS-SWIRE,20180221,20180221,2171051,13.0,5705593920,5.71 GB
4,COSMOS,20180217,20180217,2599374,5.0,10441782720,10.44 GB
5,EGS,20180217,20180217,1412613,4.0,3778816320,3.78 GB
6,ELAIS-N1,20171016,20180216,4026292,14.0,5242279680,5.24 GB
7,ELAIS-N2,20180218,20180218,1783240,9.0,2007964800,2.01 GB
8,ELAIS-S1,20180221,20180221,1655564,9.0,1970156160,1.97 GB
9,GAMA-09,20171206,20171206,12937982,62.0,19652852160,19.65 GB


In [6]:
GAVO_FOLDER = '/mnt/hedam/data_vo/'
stilts_command = 'stilts tpipe {in_file} omode=out ofmt=csv out={GAVO_FOLDER}{out_file}'

final_data = open('help_to_vo.sh', 'w+')
for field in dr1:
    final_help_product = './dmu32_{}/data/{}_{}.fits'.format(field[0], field[0], field[1])
    cigale_input = './dmu32_{}/data/{}_{}_cigale.fits'.format(field[0], field[0], field[1])

    if os.path.exists(final_help_product):
        print(final_help_product)
    
        #Test with Cigale input files
        final_data.write(stilts_command.format(in_file=final_help_product, 
                                               GAVO_FOLDER=GAVO_FOLDER, 
                                               out_file='/herschelhelp/{}_{}_all.csv \n'.format(field[0], 
                                                                                                field[1])))
    elif os.path.exists(cigale_input):
        print(cigale_input)
        final_data.write(stilts_command.format(in_file=cigale_input, 
                                               GAVO_FOLDER=GAVO_FOLDER, 
                                               out_file='/herschelhelp/{}_{}_incomplete.csv \n'.format(field[0], 
                                                                                                       field[1])))
        
    else:
        final_data.write('# No data for {} \n'.format(field[0]))
        
    #final_data.write('./dmu32_{}/data/{}_{}.fits'.format(field[0], field[0], field[1]))
    
final_data.close()

./dmu32_EGS/data/EGS_20180217_cigale.fits
./dmu32_HDF-N/data/HDF-N_20180219_cigale.fits


The out put of this notebook is a shell script which will write all the fits files to csv files in the vo folder